![BTS](https://github.com/vfp1/bts-mbds-data-science-foundations-2019/raw/master/sessions/img/Logo-BTS.jpg)

# Session 6: Decision Trees EXERCISES


### Filipa Peleja <filipa.peleja@bts.tech>
### Victor F. Pajuelo Madrigal <victor.pajuelo@bts.tech>

## Classical Data Analysis (22-02-2022)

Open this notebook in Google Colaboratory: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vfp1/bts-cda-2020/blob/master/06_DECISION_TREES/Session_6_Classical_Data_Analysis_DT_EXERCISES.ipynb)





## EXERCISE 1 - Decision Trees and Cross Validation

Follow the steps to find the best parameters on the Decision Tree classification over a Moon dataset. Try to play with the hyperparameters and reason your results with theory in hand.

### Generate a moons dataset

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

### Split the train and test

In [ ]:
from sklearn.model_selection import train_test_split

???

### Use grid search with Cross Validation
Try to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_leaf_nodes': list(range(2, 100)), 
          'min_samples_split': [2, 3, 4], 
          'criterion':['entropy', 'gini']}
          
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X, y)

### Find the best estimator for GridSearch

In [ ]:
grid_search_cv.best_estimator_

In [ ]:
grid_search_cv.best_params_

In [ ]:
# Visualize the Decision Tree (you need to have graphviz installed)
from graphviz import Source
from sklearn.tree import export_graphviz

# Call the export with Graphviz
export_graphviz(
        tree_clf,
        out_file=os.path.join(IMAGES_PATH, "iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

# Load the file in the notebook
Source.from_file(os.path.join(IMAGES_PATH, "iris_tree.dot"))

### Visualize the split and the tree growth with the code above

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

### Evaluate the accuracy with test
What did you get?

In [ ]:
from sklearn.metrics import accuracy_score

???

## EXERCISE 2 - My first random forest

You are already acquanted with the pitfalls of Decision Trees and might have an intutition of why Random Forest might be a good approach to address those pitfalls.

In this exercise you will build your own random forest from scratch. Be creative, make use of for loops or any programmnig tool that you like. Let's go:

## Part 1 - splitting the dataset to build a forest

Go on with the exercise above, use the same datraset. But this time generate 1000 subsets of the **training** dataset, with each containing 100 instances selected in a random manner.

You can code it by your own or use [Scikit-Learn ShuffleSplit class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html).

In [ ]:
# Do your splits here

## Part 2 - Train each subset!

Use the best hyperparameters for the whole moons dataset that you found in the previous exercise. **Train one Decision Tree on each subset** (the subsets you just created in Part 1). 

Evaluate the 1000 Decision trees you trained over the test set. Create an empty list, and use the method you prefer to loop through each tree, evaluating over the test set and appending the results over the empty list. Once you have your list of accuracies, calculate the mean of all of them.

This will most likely perform worse than the Decision Tree you trained with the whole train dataset, achieving not more than 80%. 

Place your final mean accuracy below.

In [ ]:
# Train each subset here and print the final mean accuracy over the 1000 trees

## Part 3 - keeping the most frequent predictions

Now we will keep only the most frequent predictions.

For each test set **INSTANCE** or **SAMPLE**, generate the predictions of the 1000 Decision trees, and keep only the most frequent ones.

For instance:

* Test $y_0$ - prediction 1
* Test $y_1$ - prediction 1
* Test $y_2$ - prediction 0
* Test $y_3$ - prediction 1
* Test $y_4$ - prediction 1

You will keep prediction = 1, as it is the most frequent. You can use SciPy's mode() function to achieve this. 

Now we have a majority-vote predictions over the whole test set, having the most common predictions for each instance over the test set. Keep this value as an array with the same shape as y_test.

In [ ]:
# Do your most frequent predictions here

## Part 4 - let's see what happens for accuracy!

Use the array that you produced above with the most common predictions for each test instance. Compare it with the y_test set using the accuracy_score method.

Basically, you should do the following:

accuracy_score(my_y_test_dataset, my_most_common_predictions_over_y_dataset)

You might need to reshape your most common predictions array so it will be the same shape as the y_test.

You should get a slightly higher accuracy, maybe up to 1.5% higher. You just trained your first Random Forest classifier!!! Congratulations! Go an have a beer!

But first, comment on why do you think that you are getting a better accuracy. Reflect on Decision Trees instabilities and why creating a Randomized Forest of Decision Trees might yield better results.

In [ ]:
# Do your final accuracy score here

## Place your reflections here: